<a href="https://colab.research.google.com/github/windyday0622/windyday/blob/main/m6_%EB%94%A5%EB%9F%AC%EB%8B%9D/%20m6_08_pytorch_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q. mnist 데이터 셋에 대하여 Pytorch를 적용하여 모델 구성 변경, 조기 학습 중단을 수행하고 Best model을 저장한 후 다시불러와서 테스트 데이터로 평가한 결과를 출력하세요.

In [ ]:
# torch.max(outputs, 1)

import torch

# 예시 출력 텐서
outputs = torch.tensor([[0.1, 0.3, 0.6], [0.2, 0.7, 0.1]])

# 가장 큰 값을 가지는 클래스 인덱스를 추출
_, predicted = torch.max(outputs, 1) # 각 행에 대해 최대값

print(predicted)  # tensor([2, 1])

tensor([2, 1])


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# MNIST 데이터셋을 위한 전처리 과정 정의
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# MNIST 데이터셋 로드
train_dataset = torchvision.datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# 데이터 로더 정의
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 아키텍처 정의
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(2880, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MyModel()

# 손실 함수 및 최적화 알고리즘 지정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 모델 훈련
best_val_loss = float('inf')
patience, trials = 5, 0
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # 검증 단계
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for inputs, labels in valloader:
                output = model(inputs)
                loss = criterion(output, labels)
                val_loss += loss.item()

        val_loss /= len(valloader)
        print(f"Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}, Val Loss: {val_loss}")

        # 검증 손실이 개선되었는지 확인하고 모델 저장
        if val_loss < best_val_loss:
            print(f'Validation loss decreased ({best_val_loss:.6f} --> {val_loss:.6f}) \t  Saving The Model')
            best_val_loss = val_loss
            trials = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            trials += 1
            if trials >= patience:
                print("Early stopping triggered")
                break

# 최고의 모델을 불러와서 평가
model.load_state_dict(torch.load('best_model.pth'))

# 모델 평가
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the 10000 test images: {accuracy} %')

Epoch 1, Train Loss: 0.003122352917989095, Val Loss: 2.30473053201716
Validation loss decreased (inf --> 2.304731) 	  Saving The Model
Epoch 1, Train Loss: 0.006184267044067383, Val Loss: 2.2941723592737886
Validation loss decreased (2.304731 --> 2.294172) 	  Saving The Model
Epoch 1, Train Loss: 0.009255009651184082, Val Loss: 2.281798168699792
Validation loss decreased (2.294172 --> 2.281798) 	  Saving The Model
Epoch 1, Train Loss: 0.01231139850616455, Val Loss: 2.268167220531626
Validation loss decreased (2.281798 --> 2.268167) 	  Saving The Model
Epoch 1, Train Loss: 0.015325480461120606, Val Loss: 2.25461801062239
Validation loss decreased (2.268167 --> 2.254618) 	  Saving The Model
Epoch 1, Train Loss: 0.018338297526041665, Val Loss: 2.2382638061300235
Validation loss decreased (2.254618 --> 2.238264) 	  Saving The Model
Epoch 1, Train Loss: 0.021316164970397948, Val Loss: 2.2201491759178484
Validation loss decreased (2.238264 --> 2.220149) 	  Saving The Model
Epoch 1, Train Los

<ipython-input-7-4e61f6ab8f83>:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Accuracy on the 10000 test images: 94.24 %


- torch: 이것은 메인 PyTorch 라이브러리입니다. 여기에는 GPU를 통한 가속 텐서 계산 지원, 신경망 훈련을 용이하게 하는 자동 차별화, 모델 구축 및 훈련을 위한 다양한 유틸리티가 포함됩니다.
- torch.nn: 레이어, 활성화 함수, 손실 함수와 같은 신경망의 구성 요소를 제공하는 PyTorch의 하위 모듈입니다. 신경망의 아키텍처를 정의하는 데 필수적입니다.
- torch.nn.function: 이 모듈에는 torch.nn 레이어에서 사용되는 기능이 포함되어 있습니다. 입력 데이터 및 가중치에 이러한 함수를 직접 사용할 수 있으므로 일부 작업에 더 많은 유연성을 제공합니다. 여기에는 활성화, 손실 계산 및 상태(즉, 가중치)를 유지하지 않는 다양한 기타 작업을 위한 함수가 포함됩니다.
- torch.optim: 이 하위 모듈은 SGD(Stochastic Gradient Descent), Adam 등과 같은 신경망 훈련을 위한 최적화 알고리즘을 제공합니다. 이러한 최적화 프로그램은 계산된 기울기를 기반으로 네트워크의 가중치를 업데이트하는 데 사용됩니다.
- torchvision: 이미지 데이터 작업을 위한 유틸리티를 제공하는 PyTorch 프로젝트의 패키지입니다. 여기에는 사전 정의된 데이터세트(예: MNIST, CIFAR10, FashionMNIST), 모델 아키텍처(예: ResNet, AlexNet) 및 전처리를 위한 일반적인 이미지 변환이 포함됩니다.
- torchvision.transforms: 일반적인 이미지 변환을 제공하는 torchvision 내의 모듈입니다. 이는 이미지를 신경망에 공급하기 전에 데이터 증대 및 이미지 전처리에 사용될 수 있습니다. 예로는 크기 조정, 정규화, 텐서로 변환 등이 있습니다.
- SubsetRandomSampler: 교체 없이 데이터세트에서 요소를 무작위로 샘플링하는 데 사용되는 도구입니다. 데이터 세트를 훈련 및 검증/테스트 세트로 분할하거나 모델 훈련을 위해 사용자 정의 데이터 샘플링 전략을 구현하려는 경우에 특히 유용

Q. PyTorch를 사용하여 FashionMNIST 데이터세트에 대한 분류 모델링 및 평가를 다음과 같은 단계로 수행하세요.
- 1단계: 신경망 모델 정의
- 2단계: FashionMNIST 데이터셋 로드
- 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
- 4단계: 조기 종료를 포함한 모델 학습 및 best model 저장
- 5단계: best model을 로드하고 테스트 데이터셋으로 평가

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

# 1단계: 신경망 모델 정의
# Net 클래스는 nn.Module을 상속받아 만들어진 사용자 정의 신경망 모델로, FashionMNIST 데이터셋의 이미지 분류를 위해 설계
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 첫 번째 합성곱 레이어: 입력 채널 1개, 출력 채널 6개, 커널 크기 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 두 번째 합성곱 레이어: 입력 채널 6개, 출력 채널 16개, 커널 크기 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        # 전결합 레이어
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 16개의 채널과 4x4 이미지 크기
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 첫 번째 합성곱, ReLU 활성화, 맥스 풀링
        x = self.pool(F.relu(self.conv1(x)))
        # 두 번째 합성곱, ReLU 활성화, 맥스 풀링
        x = self.pool(F.relu(self.conv2(x)))
        # 입력 데이터를 1차원으로 펼침
        x = self.flatten(x)
        # x = x.view(-1, 16 * 4 * 4)
        # 전결합 레이어와 ReLU 활성화 함수 적용
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 마지막 레이어 출력
        x = self.fc3(x)
        return x

# 2단계: FashionMNIST 데이터셋 로드
# torchvision 라이브러리를 사용하여 FashionMNIST 데이터셋을 다운로드하고, 데이터를 전처리하기 위한 변환(transform)을 설정하는 과정

# transforms.Compose는 여러 전처리 단계를 하나로 묶어주는 역할
# transforms.ToTensor(): 이미지를 PyTorch 텐서로 변환. 이미지의 픽셀 값 범위가 0에서 255 사이의 정수에서 0.0에서 1.0 사이의 부동소수점으로 변경
# 모든 채널의 평균을 0.5로, 표준편차를 0.5로 설정합니다. 이는 데이터의 범위를 대략적으로 -1.0에서 1.0 사이로 조정
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# FashionMNIST 데이터셋을 다운로드하고, 지정된 변환을 적용하여 데이터를 준비하는 함수
train_val_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 및 검증 분할을 위한 데이터 인덱스 생성
dataset_size = len(train_val_dataset) # 훈련 및 검증 데이터셋의 전체 크기
indices = list(range(dataset_size)) # 데이터셋 내의 모든 샘플에 대한 인덱스를 포함
validation_split = 0.1 # 검증 세트로 사용될 데이터의 비율
split = int(np.floor(validation_split * dataset_size)) # 검증 세트의 크기를 계산
np.random.shuffle(indices) # 훈련 및 검증 세트가 데이터셋의 특정 부분에 치우치지 않도록 하기 위함
train_indices, val_indices = indices[split:], indices[:split] # 섞인 인덱스를 사용하여 훈련 세트와 검증 세트의 인덱스를 분할

# PT 데이터 샘플러 및 로더 생성
# 훈련 세트와 검증 세트에 대한 데이터 로더를 설정하고, 테스트 세트에 대한 데이터 로더를 별도로 설정하는 과정입니다.
# 이러한 데이터 로더들은 모델 학습, 검증, 테스트 과정에서 배치 단위로 데이터를 로드하는 데 사용
train_sampler = SubsetRandomSampler(train_indices) # train_indices에 해당하는 훈련 데이터의 인덱스를 무작위로 샘플링하는 샘플러를 생성
val_sampler = SubsetRandomSampler(val_indices) # val_indices에 해당하는 검증 데이터의 인덱스로부터 데이터를 무작위로 샘플링하는 샘플러를 생성
# 배치 크기 4로 로드하는 훈련 데이터 로더를 생성(# 데이터의 무작위 샘플링을 수행)
trainloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=train_sampler)
valloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=val_sampler)
# shuffle=False 인자를 통해 셔플링 없이 순서대로 데이터를 로드. 테스트 과정에서는 데이터의 순서가 결과에 영향을 미치지 않으므로 셔플링을 수행하지 않습니다.
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

# 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
net = Net() #  클래스의 인스턴스를 생성하여 net 변수에 할당
criterion = nn.CrossEntropyLoss() # 멀티클래스 분류 문제에서 널리 사용되는 손실 함수로, 모델의 예측값과 실제 타겟값 사이의 차이를 측정
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 모멘텀은 최적화 과정에서 이전 그래디언트의 방향을 고려, 파라미터 업데이트 시 관성을 부여

# 4단계: 조기 종료를 포함한 모델 학습
patience = 5 # 검증 손실이 개선되지 않을 때, 훈련을 계속 진행하기 전에 기다릴 에폭 수를 의미
patience_counter = 0
best_val_loss = np.Inf # 최고의 검증 손실 값을 무한대로 초기화. 훈련 과정에서 검증 손실이 이전에 기록된 최소 손실보다 낮아지면 업데이트되는 값

# 모델을 에폭 단위로 반복 훈련시키면서, 각 배치의 손실을 계산하고 모델 파라미터를 업데이트하는 기본적인 훈련 과정을 구현
for epoch in range(20):  # 데이터셋을 여러 번 반복
    net.train()  # 모델을 학습 모드로 설정. 이는 모델 내의 특정 레이어(예: 드롭아웃, 배치 정규화 등)가 훈련 시와 평가 시 다르게 동작해야 할 때 필요
    running_loss = 0.0 # 현재 에폭의 총 손실을 계산하기 위해 실행 손실을 0으로 초기화
    for i, data in enumerate(trainloader, 0): # '0'은 열거의 시작 인덱스를 지정
        inputs, labels = data
        optimizer.zero_grad() # 최적화를 수행하기 전에 모델의 그래디언트를 0으로 초기화
        outputs = net(inputs) # 현재 배치의 입력 데이터를 모델에 전달하여 예측값을 계산
        loss = criterion(outputs, labels) # 모델의 예측값과 실제 레이블 간의 손실을 계산
        loss.backward() # 손실 함수의 그래디언트를 역전파합니다. 이 과정에서 모델 파라미터에 대한 손실의 미분값이 계산
        optimizer.step() # 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트
        running_loss += loss.item() # 현재 배치의 손실을 실행 손실에 누적. 이를 통해 전체 에폭의 평균 손실을 계산

    # 검증 단계
    net.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0 # 검증 손실을 계산하기 위한 변수를 0으로 초기화
    with torch.no_grad(): # 이 블록 내에서는 그래디언트 계산을 비활성화. 평가 모드에서는 모델을 업데이트하지 않으므로, 그래디언트를 계산할 필요가 없습니다
        for inputs, labels in valloader: # 검증 데이터 로더(valloader)에서 배치 단위로 데이터를 로드하여 반복
            outputs = net(inputs)
            loss = criterion(outputs, labels) # criterion은 손실 함수로, 모델의 성능을 측정하는 기준
            val_loss += loss.item() # 누적된 검증 손실을 검증 데이터 배치의 총 수로 나누어 평균 검증 손실을 계산
    running_loss /= len(trainloader)
    val_loss /= len(valloader)
    print(f'에폭 {epoch + 1}, 훈련 손실: {running_loss}, 검증 손실: {val_loss}')

    # 조기 종료 체크
    if val_loss < best_val_loss: # 현재 에폭에서 계산된 검증 손실(val_loss)이 이전에 기록된 최소 검증 손실(best_val_loss)보다 낮은지 확인
        print(f'검증 손실이 감소하였습니다. ({best_val_loss:.6f} 에서 {val_loss:.6f}로). 모델 저장 중...')
        torch.save(net.state_dict(), '/content/drive/MyDrive/kdt_240424/m6_dl/data/model/best_model.pth')
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('조기 종료 발동.')
            break

# 5단계: 최고의 모델을 로드하고 테스트 데이터셋으로 평가
# torch.load 함수는 지정된 경로에서 모델 파라미터를 불러오며, load_state_dict 메서드를 사용하여 이 파라미터를 현재 net 모델에 로드
net.load_state_dict(torch.load('/content/drive/MyDrive/kdt_240424/m6_dl/data/model/best_model.pth'))
correct = 0 # 정확히 분류된 샘플의 수를 세기 위한 변수
total = 0 # 테스트셋의 전체 샘플 수를 세기 위한 변수
with torch.no_grad():
    for data in testloader: # 테스트 데이터셋을 배치 단위로 순회
        images, labels = data
        outputs = net(images) # 현재 배치의 이미지를 모델에 전달하여 예측값을 계산
        _, predicted = torch.max(outputs.data, 1) # torch.max는 각 예측에 대한 최대값과 그 위치(인덱스)를 반환. 위치만 필요하므로 _를 사용하여 최대값은 무시
        total += labels.size(0) # labels.size(0)는 현재 배치의 크기(샘플 수)
        correct += (predicted == labels).sum().item() # 일치하는 경우의 수를 텐서 형태로 반환하며, .item()으로 이를 파이썬의 스칼라 값으로 변환

print(f'10000개의 테스트 이미지에 대한 네트워크의 정확도: {100 * correct / total} %')


에폭 1, 훈련 손실: 0.653004357415329, 검증 손실: 0.497642044117713
검증 손실이 감소하였습니다. (inf 에서 0.497642로). 모델 저장 중...
에폭 2, 훈련 손실: 0.3832080303337781, 검증 손실: 0.3661725013044973
검증 손실이 감소하였습니다. (0.497642 에서 0.366173로). 모델 저장 중...
에폭 3, 훈련 손실: 0.3320615399670671, 검증 손실: 0.32115839002634555
검증 손실이 감소하였습니다. (0.366173 에서 0.321158로). 모델 저장 중...
에폭 4, 훈련 손실: 0.30448877325246115, 검증 손실: 0.3230597239609827
에폭 5, 훈련 손실: 0.2839316014807367, 검증 손실: 0.3175863423420985
검증 손실이 감소하였습니다. (0.321158 에서 0.317586로). 모델 저장 중...
에폭 6, 훈련 손실: 0.2691086102266032, 검증 손실: 0.3012078336844649
검증 손실이 감소하였습니다. (0.317586 에서 0.301208로). 모델 저장 중...
에폭 7, 훈련 손실: 0.25550138932721994, 검증 손실: 0.30303460587284403
에폭 8, 훈련 손실: 0.24571981320756842, 검증 손실: 0.3064329963752328
에폭 9, 훈련 손실: 0.23613766410586817, 검증 손실: 0.29779855652345333
검증 손실이 감소하였습니다. (0.301208 에서 0.297799로). 모델 저장 중...
에폭 10, 훈련 손실: 0.22580871239718425, 검증 손실: 0.28825473328034196
검증 손실이 감소하였습니다. (0.297799 에서 0.288255로). 모델 저장 중...
에폭 11, 훈련 손실: 0.21562310846693059, 검증 손실: 0

<ipython-input-2-a3c9f82679c0>:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('/content/drive/MyDrive/kdt_240424/m6_dl/data/model/best_mod

10000개의 테스트 이미지에 대한 네트워크의 정확도: 90.07 %


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 1단계: 신경망 모델 정의
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 2단계: FashionMNIST 데이터셋 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

# 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
model = FashionMNISTModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4단계: 조기 종료를 포함한 모델 학습 및 best model 저장
best_val_loss = float('inf')
patience, trials = 5, 0
num_epochs = 20
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(trainloader)

    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for inputs, labels in valloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(valloader)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        trials = 0
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print("Saved best model")
    else:
        trials += 1
        if trials >= patience:
            print("Early stopping triggered")
            break

# 5단계: best model을 로드하고 테스트 데이터셋으로 평가
model.load_state_dict(torch.load('best_model.pth'))

testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test dataset: {accuracy:.2f}%")

100%|██████████| 26421880/26421880 [00:06<00:00, 4330859.46it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 215619.44it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3914357.75it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4951221.51it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Epoch 1, Train Loss: 0.453330, Val Loss: 0.323051
Saved best model
Epoch 2, Train Loss: 0.283177, Val Loss: 0.281939
Saved best model
Epoch 3, Train Loss: 0.237202, Val Loss: 0.248629
Saved best model
Epoch 4, Train Loss: 0.203577, Val Loss: 0.246991
Saved best model
Epoch 5, Train Loss: 0.179454, Val Loss: 0.248894
Epoch 6, Train Loss: 0.153368, Val Loss: 0.228347
Saved best model
Epoch 7, Train Loss: 0.132176, Val Loss: 0.243469
Epoch 8, Train Loss: 0.111976, Val Loss: 0.236311
Epoch 9, Train Loss: 0.093725, Val Loss: 0.264679
Epoch 10, Train Loss: 0.076867, Val Loss: 0.286979
Epoch 11, Train Loss: 0.062992, Val Loss: 0.300919
Early stopping triggered


<ipython-input-8-84fd25f88ca8>:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Accuracy on the test dataset: 91.49%


Task1_0829. 가상 데이터 생성 (generate_data 함수) 후 모델링 및 평가하세요
- generate_data 함수는 1000개의 랜덤 시퀀스 데이터를 생성합니다.
  - vocab_size: 시퀀스에 사용할 어휘의 크기를 설정합니다. 여기서는 100개의 단어를 사용합니다.
  - data: 각 시퀀스는 seq_length=10으로 설정된 10개의 정수(단어 인덱스)로 구성됩니다.
  - labels: 각 시퀀스에 대해 0 또는 1의 이진 레이블을 무작위로 할당합니다.

- LSTM 기반 분류 모델을 정의하고, 가상 데이터로 학습 및 검증을 수행합니다.
- 조기 종료를 통해 학습 중 성능이 더 이상 개선되지 않을 때 학습을 중단합니다.
최종적으로 테스트 데이터로 최고 성능의 모델을 평가합니다.

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels

data, labels = generate_data()

#### nn.Embedding(vocag_size,embed_dim)
- 텍스트 데이터(LSTM 모델): 텍스트 데이터는 이산적인 정수로 표현되므로, 이 정수들을 고차원 벡터로 매핑하는 nn.Embedding 계층이 필요합니다. 이 임베딩 계층은 단어 간의 의미적 유사성을 학습하는 데 유용합니다.

- 이미지 데이터(CNN 모델): 이미지 데이터는 이미 공간적 구조를 가진 연속적인 값(픽셀)으로 표현되므로, 임베딩 계층이 필요하지 않습니다. 대신, 합성곱 계층이 이미지의 패턴을 학습하는 데 사용됩니다.

LSTM 모델의 경우 텍스트 데이터의 정수 인덱스를 벡터로 변환하기 위해 nn.Embedding이 필요하지만, CNN 모델에서는 이미지 데이터를 처리하는 데 이미 직접적인 합성곱 연산이 사용되므로 임베딩 계층이 필요하지 않습니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels

data, labels = generate_data()

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))
    return data, labels

data, labels = generate_data()

# 데이터셋 및 데이터로더 생성
dataset = TensorDataset(data, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# LSTM 기반 분류 모델 정의
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = output[:, -1, :]
        output = self.fc(output)
        return output

# 모델 초기화
vocab_size = 100
embedding_dim = 32
hidden_dim = 64
output_dim = 2

model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)

# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 및 검증 함수 정의
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(dataloader), correct / total

# 학습 및 검증 수행
best_val_loss = float('inf')
patience = 5
trials = 0
num_epochs = 20

for epoch in range(num_epochs):
    train_loss = train(model, train_dataloader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, val_dataloader, criterion)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, Val Acc: {val_acc:.6f}")

    if val_loss < best_val_loss:
        trials = 0
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print("Saved best model")
    else:
        trials += 1
        if trials >= patience:
            print("Early stopping triggered")
            break

# 테스트 데이터로 최고 성능의 모델 평가
test_data, test_labels = generate_data(num_samples=200)
test_dataset = TensorDataset(test_data, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_acc = evaluate(model, test_dataloader, criterion)
print(f"Test Loss: {test_loss:.6f}, Test Acc: {test_acc:.6f}")

Epoch 1, Train Loss: 0.693183, Val Loss: 0.698095, Val Acc: 0.470000
Saved best model
Epoch 2, Train Loss: 0.684583, Val Loss: 0.696612, Val Acc: 0.480000
Saved best model
Epoch 3, Train Loss: 0.679973, Val Loss: 0.695801, Val Acc: 0.495000
Saved best model
Epoch 4, Train Loss: 0.671451, Val Loss: 0.698579, Val Acc: 0.470000
Epoch 5, Train Loss: 0.663889, Val Loss: 0.708790, Val Acc: 0.465000
Epoch 6, Train Loss: 0.654097, Val Loss: 0.708813, Val Acc: 0.490000
Epoch 7, Train Loss: 0.640969, Val Loss: 0.727367, Val Acc: 0.460000
Epoch 8, Train Loss: 0.629210, Val Loss: 0.737537, Val Acc: 0.470000
Early stopping triggered
Test Loss: 0.700666, Test Acc: 0.555000


<ipython-input-2-dd6ac8b42094>:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels

data, labels = generate_data()

# 텐서 데이터셋 및 데이터 로더 생성
dataset = TensorDataset(data, labels)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - (train_size + val_size)
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# LSTM 모델 클래스 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        lstm_out, (hidden, _) = self.lstm(embedded)
        hidden = hidden[-1,:,:]
        return self.fc(hidden)

model = LSTMModel(vocab_size=100, embed_dim=50, hidden_dim=100, output_dim=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

# 훈련 함수
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data).squeeze(1)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# 평가 함수
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data, labels in data_loader:
            outputs = model(data).squeeze(1)
            loss = criterion(outputs, labels.float())
            total_loss += loss.item()
            predictions = torch.round(torch.sigmoid(outputs))
            correct_predictions = (predictions == labels.unsqueeze(1)).float()
            total_accuracy += correct_predictions.sum().item()
    return total_loss / len(data_loader), total_accuracy / len(data_loader.dataset)

# 조기 종료 로직을 포함한 훈련 및 검증 과정
best_val_loss = float('inf') # float('inf')는 파이썬에서 양의 무한대를 나타내는 방식
patience = 3
trials = 0

for epoch in range(20):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trials += 1
        if trials >= patience:
            print("조기 종료 발생")
            break

# 테스트 데이터로 최고 모델 평가
model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_accuracy = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Epoch 1, Train Loss: 0.6952, Val Loss: 0.6940, Val Accuracy: 15.1200
Epoch 2, Train Loss: 0.6809, Val Loss: 0.6927, Val Accuracy: 15.4267
Epoch 3, Train Loss: 0.6671, Val Loss: 0.6936, Val Accuracy: 15.3467
Epoch 4, Train Loss: 0.6478, Val Loss: 0.7040, Val Accuracy: 15.3867
Epoch 5, Train Loss: 0.6206, Val Loss: 0.7172, Val Accuracy: 15.3333
조기 종료 발생
Test Loss: 0.6872, Test Accuracy: 15.0800


<ipython-input-3-c109df005c34>:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
